In [22]:

import urllib
import xml.etree.ElementTree as ET
import urllib
import re
import io, random
import pandas as pd
import time
from datetime import datetime
from sklearn.svm import SVR
import datetime


In [23]:
df=pd.read_csv("merged_final.csv")

In [24]:
df.head()

,Unnamed: 0,DATE_TIME,PLANT_ID,SCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,PLANT_ID.1,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,0,15-05-2020 00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0
1,1,15-05-2020 00:00,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0
2,2,15-05-2020 00:00,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0
3,3,15-05-2020 00:00,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0
4,4,15-05-2020 00:00,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0


In [44]:
def pred(x, y, x_test):
    svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
    y_rbf = svr_rbf.fit(x, np.ravel(y,order='C'))
    return svr_rbf.predict(x_test)

 

def AMWR(df):
    x_total = df[["AMBIENT_TEMPERATURE","MODULE_TEMPERATURE","IRRADIATION"]].to_numpy().tolist()
    y_total = df[["DAILY_YIELD"]].to_numpy().tolist()
    x = x_total[:len(df)-1]
    y = y_total[:len(df)-1]
    x_test = x_total[len(df)-1:]
    y_test = y_total[len(df)-1:]
    y_pred = pred(x,y,x_test)
    if(y_test!=0):
        MAPE_list.append([y_test[0][0],y_pred[0]])

In [45]:
def dataprocess(df,dt_start):
    dt_end=dt_start+datetime.timedelta(0,TrainingWindow*time_sampling)
    #print(dt_start,dt_end)
    #print(sensor_list)
    #df = df.set_index(['DATE_TIME'])
    #print(df.loc[dt_start:dt_end])
    df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'])
    mask = (df['DATE_TIME'] > dt_start) & (df['DATE_TIME'] <= dt_end)
    df_proc=df.loc[mask]
    for i in sensor_list:
        msk=(df_proc['SCE_KEY']==i[0])
        df_proc_sen=df_proc.loc[msk]
        AMWR(df_proc_sen)
    
    

In [56]:
#constant parameters
TrainingWindow = 30    #window size is found from historical data
PredictionWindow = 3   #prediction window/prediction horizon
time_sampling = 900 #Should be equal to data refreshing time
sensor_list=df[["SCE_KEY"]].to_numpy().tolist()
sensor_list=sensor_list[0:20]

#main function
if __name__ == '__main__':
    MAPE_list = list()
    date=datetime.datetime(2020, 5, 25, 6, 15)
    dataprocess(df,date)
    print(MAPE_list)
    MAPE_npArray = np.array(MAPE_list)
    print(MAPE_npArray[:,0])
    mape = np.mean(np.abs((MAPE_npArray[:,0] - MAPE_npArray[:,1])/MAPE_npArray[:,0]))*100
    print("MAPE is: "+ str(mape))

[[5951.0, 5001.349273357164], [6507.428571, 5314.106613749326], [6520.285714, 5324.146679710648], [6311.0, 5204.504597943816], [6447.142857, 5246.926925260154], [6482.142857, 5295.952421133673], [6388.714286, 5260.934964068658], [6383.285714, 5223.72614074077], [6235.285714, 5127.083185720567], [6600.857143, 5363.0920782623425], [5829.142857, 4933.92332268176], [6337.0, 5194.78450438123], [5655.857143, 4735.675131332344], [6259.714286, 5112.425899817844], [6264.125, 5151.030559757643], [6282.142857, 5167.174161880497], [6321.142857, 5162.472531164948], [6371.428571, 5204.672972631726], [6337.571429, 5201.7494185161395], [6018.714286, 5014.948750760932]]
[5951.       6507.428571 6520.285714 6311.       6447.142857 6482.142857
 6388.714286 6383.285714 6235.285714 6600.857143 5829.142857 6337.
 5655.857143 6259.714286 6264.125    6282.142857 6321.142857 6371.428571
 6337.571429 6018.714286]
MAPE is: 17.708557447865406


In [58]:
#constant parameters
TrainingWindow = 10    #window size is found from historical data
PredictionWindow = 3   #prediction window/prediction horizon
time_sampling = 900 #Should be equal to data refreshing time
sensor_list=df[["SCE_KEY"]].to_numpy().tolist()
sensor_list=sensor_list[0:20]

#main function
if __name__ == '__main__':
    for i in range(1,90):
        TrainingWindow+=1
        MAPE_list = list()
        date=datetime.datetime(2020, 5, 25, 6, 15)
        dataprocess(df,date)
        MAPE_npArray = np.array(MAPE_list)
        mape = np.mean(np.abs((MAPE_npArray[:,0] - MAPE_npArray[:,1])/MAPE_npArray[:,0]))*100
        print("MAPE is: "+ str(mape)+"Training window is: "+str(TrainingWindow) )

MAPE is: 39.176630046371116Training window is: 11
MAPE is: 43.81496842026997Training window is: 12
MAPE is: 22.654531463046457Training window is: 13
MAPE is: 34.728435530541425Training window is: 14
MAPE is: 27.080841481674067Training window is: 15
MAPE is: 27.434919390564332Training window is: 16
MAPE is: 36.94354718572881Training window is: 17
MAPE is: 48.72353532171143Training window is: 18
MAPE is: 37.78776987669959Training window is: 19
MAPE is: 30.63274223547757Training window is: 20
MAPE is: 21.55357131855311Training window is: 21
MAPE is: 32.654171295743645Training window is: 22
MAPE is: 24.985452071141587Training window is: 23
MAPE is: 25.377493763357418Training window is: 24
MAPE is: 19.283433479109163Training window is: 25
MAPE is: 35.97811127120803Training window is: 26
MAPE is: 27.796053557330154Training window is: 27
MAPE is: 28.662436638343237Training window is: 28
MAPE is: 20.325535125106626Training window is: 29
MAPE is: 17.708557447865406Training window is: 30
MAPE is

<ipython-input-58-9aaa14c787b2>:16: RuntimeWarning: divide by zero encountered in true_divide
  mape = np.mean(np.abs((MAPE_npArray[:,0] - MAPE_npArray[:,1])/MAPE_npArray[:,0]))*100
<ipython-input-58-9aaa14c787b2>:16: RuntimeWarning: divide by zero encountered in true_divide
  mape = np.mean(np.abs((MAPE_npArray[:,0] - MAPE_npArray[:,1])/MAPE_npArray[:,0]))*100
<ipython-input-58-9aaa14c787b2>:16: RuntimeWarning: divide by zero encountered in true_divide
  mape = np.mean(np.abs((MAPE_npArray[:,0] - MAPE_npArray[:,1])/MAPE_npArray[:,0]))*100
<ipython-input-58-9aaa14c787b2>:16: RuntimeWarning: divide by zero encountered in true_divide
  mape = np.mean(np.abs((MAPE_npArray[:,0] - MAPE_npArray[:,1])/MAPE_npArray[:,0]))*100


MAPE is: infTraining window is: 73
MAPE is: infTraining window is: 74
MAPE is: infTraining window is: 75


<ipython-input-58-9aaa14c787b2>:16: RuntimeWarning: divide by zero encountered in true_divide
  mape = np.mean(np.abs((MAPE_npArray[:,0] - MAPE_npArray[:,1])/MAPE_npArray[:,0]))*100
<ipython-input-58-9aaa14c787b2>:16: RuntimeWarning: divide by zero encountered in true_divide
  mape = np.mean(np.abs((MAPE_npArray[:,0] - MAPE_npArray[:,1])/MAPE_npArray[:,0]))*100
<ipython-input-58-9aaa14c787b2>:16: RuntimeWarning: divide by zero encountered in true_divide
  mape = np.mean(np.abs((MAPE_npArray[:,0] - MAPE_npArray[:,1])/MAPE_npArray[:,0]))*100
<ipython-input-58-9aaa14c787b2>:16: RuntimeWarning: divide by zero encountered in true_divide
  mape = np.mean(np.abs((MAPE_npArray[:,0] - MAPE_npArray[:,1])/MAPE_npArray[:,0]))*100


MAPE is: infTraining window is: 76
MAPE is: infTraining window is: 77
MAPE is: infTraining window is: 78
MAPE is: infTraining window is: 79


...

MAPE is: infTraining window is: 87
MAPE is: infTraining window is: 88
MAPE is: infTraining window is: 89


<ipython-input-58-9aaa14c787b2>:16: RuntimeWarning: divide by zero encountered in true_divide
  mape = np.mean(np.abs((MAPE_npArray[:,0] - MAPE_npArray[:,1])/MAPE_npArray[:,0]))*100
<ipython-input-58-9aaa14c787b2>:16: RuntimeWarning: divide by zero encountered in true_divide
  mape = np.mean(np.abs((MAPE_npArray[:,0] - MAPE_npArray[:,1])/MAPE_npArray[:,0]))*100
<ipython-input-58-9aaa14c787b2>:16: RuntimeWarning: divide by zero encountered in true_divide
  mape = np.mean(np.abs((MAPE_npArray[:,0] - MAPE_npArray[:,1])/MAPE_npArray[:,0]))*100


MAPE is: infTraining window is: 90
MAPE is: infTraining window is: 91
MAPE is: infTraining window is: 92


<ipython-input-58-9aaa14c787b2>:16: RuntimeWarning: divide by zero encountered in true_divide
  mape = np.mean(np.abs((MAPE_npArray[:,0] - MAPE_npArray[:,1])/MAPE_npArray[:,0]))*100
<ipython-input-58-9aaa14c787b2>:16: RuntimeWarning: divide by zero encountered in true_divide
  mape = np.mean(np.abs((MAPE_npArray[:,0] - MAPE_npArray[:,1])/MAPE_npArray[:,0]))*100
<ipython-input-58-9aaa14c787b2>:16: RuntimeWarning: divide by zero encountered in true_divide
  mape = np.mean(np.abs((MAPE_npArray[:,0] - MAPE_npArray[:,1])/MAPE_npArray[:,0]))*100


MAPE is: infTraining window is: 93
MAPE is: infTraining window is: 94
MAPE is: infTraining window is: 95
MAPE is: 92626.86712315484Training window is: 96
MAPE is: 13659.049872821959Training window is: 97
MAPE is: 9411.59297592781Training window is: 98
MAPE is: 6193.650967959546Training window is: 99
